In [1]:
from semantics import onto
import pyiron_nodes as pn
from pyiron_workflow import Workflow
from pyironflow.pyironflow import PyironFlow, GUILayout

Press "Load Workflow" after executing the following cell.</br>
Change the numbers in gui_layout to fit your screen. </br>
Click on a node and press "Run" to execute

In [2]:
wf = Workflow("concept_dict_wf")
gui_layout = GUILayout()
gui_layout.flow_widget_height = 1100
gui_layout.flow_widget_width = 1650
gui_layout.output_widget_width = 600
pf = PyironFlow([wf], root_path="pyiron_nodes", gui_layout=gui_layout)
pf.gui

In [73]:
dc = wf_gui.SubstituteImpurityAtom.outputs.structure_dataclass.value
function = wf_gui.RestrictedNode.node_function
check_restrictions(function, dc)

Restriction target: "crystal_defect"
Incoming class(es):  ['Vacancy', 'SubstitutionalImpurity']
Checking mode: all
Restriction class:  PointDefect

Result:  True



In [78]:
from semantikon.converter import parse_input_args, parse_metadata
parse_metadata(dc.chemical_species.__class__.atoms)

{'units': None,
 'label': 'atoms',
 'triples': None,
 'uri': semantics.onto.Atom,
 'shape': None,
 'restrictions': None}

In [74]:
def check_restrictions(function, dc):
    from semantikon.converter import parse_input_args, parse_metadata
    for k in parse_input_args(function).keys():
        class_res = []
        val_res = []
        try:
            class_res = parse_input_args(function)[k]['class_restrictions']
        except:
            pass
        try:
            val_res = parse_input_args(function)[k]['value_restrictions']
        except:
            pass
        class_restriction(class_res, k, dc)

In [79]:
def class_restriction(class_res, k, dc):
    for keywords in class_res:
        checking_mode = keywords[0]
        restriction_path = [str(i) for i in keywords[1:-2]]
        restriction_key = '.'.join(restriction_path)
        restriction_class = keywords[-1]

        incoming_class_objects = []
        if isinstance(eval("dc." + restriction_key), list):
            for item in eval("dc." + restriction_key):
                incoming_class_objects.append(item.__class__._semantikon_metadata['uri'])
        else:
            incoming_class_objects.append(dc.__class__._semantikon_metadata['uri'])

        if checking_mode == 'any':
            check = False
            for obj_class in incoming_class_objects:
                obj = obj_class()
                if isinstance(obj, restriction_class): 
                    check = True
        elif checking_mode == 'all':
            check = True
            for obj_class in incoming_class_objects:
                obj = obj_class()
                if not isinstance(obj, restriction_class):
                    check = False

        incoming_class_objects_names = []
        
        for item in incoming_class_objects:
            incoming_class_objects_names.append(item.__name__)

        print('Restriction target: "' + restriction_key + '"')
        print('Incoming class(es): ', incoming_class_objects_names)
        print('Checking mode: ' + checking_mode)
        print ('Restriction class: ', restriction_class.__name__)
        print('')
        print('Result: ', check)
        print('')

        return check

In [26]:
wf_gui = pf.get_workflow()
wf_gui()

{
  dataclass: AtomisticSample 
  uri: http://purls.helmholtz-metadaten.de/cmso/AtomicScaleSample 
}
--------------------------------------------------
  {
    field_name: class_object 
    uri: http://purl.obolibrary.org/obo/IAO_0000030 
    value: python id 139896436247056 
    units: None 
    data_type:  Atoms 
  }
--------------------------------------------------
  {
    field_name: simulation_cell 
    uri: http://purls.helmholtz-metadaten.de/cmso/SimulationCell 
    value: python id 139896823637792 
    units: None 
    data_type:  SampleSimulationCell 
  }
    {
      sub_field_name: simulation_cell_lengths 
      uri: http://purls.helmholtz-metadaten.de/cmso/Length 
      value:  [2.87, 2.87, 2.87] 
      units: ANGSTROM 
      data_type:  list 
    }
    {
      sub_field_name: simulation_cell_vectors 
      uri: http://purls.helmholtz-metadaten.de/cmso/SimulationCellVector 
      value:  [array([2.87, 0.  , 0.  ]), array([0.  , 2.87, 0.  ]), array([0.  , 0.  , 2.87])] 
    

{'VisualizeStructure__view': NGLWidget(),
 'VisualizeStructure0__view': NGLWidget(),
 'VisualizeStructure1__view': NGLWidget(),
 'VisualizeStructure2__view': NGLWidget(),
 'VisualizeStructure3__view': NGLWidget(),
 'RestrictedNode__result': 'You may pass.',
 'VisualizeDataclass__Done': 'Done',
 'VisualizeDataclass0__Done': 'Done',
 'VisualizeDataclass1__Done': 'Done',
 'VisualizeDataclass2__Done': 'Done',
 'VisualizeDataclass3__Done': 'Done'}

In [155]:
for k_1,v_1 in wf_gui.graph_as_dict['edges']['data']:
    pass#DCViz(wf_gui.graph_as_dict['edges']['data'][(k_1,v_1)][0].value)

In [176]:
from pyiron_nodes.visualize_dataclass import DCViz
from dataclasses import fields
print("\nWORFKLOW: " + wf_gui.label + "\n")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("                     NODES                      ")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
for k in wf_gui.children:
    print("{")
    print('  ' + "node:" + k)
    print('  ' + "node_class:", wf_gui.children[k].__class__.__name__)
    try:
        print('  ' + "node_semantic_type:", wf_gui.children[k].__class__.node_function._semantikon_metadata['uri']().IRI)
    except:
        pass
    input_ports = []
    output_ports = []
    for v in wf_gui.children[k].inputs:
        input_ports.append(v.label)
    for v in wf_gui.children[k].outputs:
        output_ports.append(v.label)
    print('  ' + "input_ports: { [" )
    for item in input_ports:
        print('  ' + '  ' + item)
    print('  ' +  "] }" )
    print('  ' + "output_ports: { [" )
    for item in output_ports:
        print('  ' + '  ' + item)
    for v in wf_gui.children[k].outputs:
        try:
            v.value.__class__._semantikon_metadata
            DCViz(v.value)
        except:
            pass
    print('  ' + "] }" )
    print("}")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("                     EDGES                      ")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
for k_1,v_1 in wf_gui.graph_as_dict['edges']['data']:
    print("{")
    print("  " + "source: " + k_1)
    print("  " + "target: " + v_1)
    try:
        wf_gui.graph_as_dict['edges']['data'][(k_1,v_1)][0].value.__class__._semantikon_metadata
        #DCViz(wf_gui.graph_as_dict['edges']['data'][(k_1,v_1)][0].value)
    except:
        pass
    print("}")


WORFKLOW: concept_dict_wf

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                     NODES                      
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
{
  node:CreateStructure
  node_class: CreateStructure
  node_semantic_type: https://id-from-pmdco-pending/StructureCreationNode
  input_ports: { [
    pr_name
    element
    bravais_lattice
    a
    c
    cubic
  ] }
  output_ports: { [
    structure_dataclass
{
  dataclass: AtomisticSample 
  uri: http://purls.helmholtz-metadaten.de/cmso/AtomicScaleSample 
}
--------------------------------------------------
  {
    field_name: class_object 
    uri: http://purl.obolibrary.org/obo/IAO_0000030 
    value: python id 140179076886320 
    units: None 
    data_type:  Atoms 
  }
--------------------------------------------------
  {
    field_name: simulation_cell 
    uri: http://purls.helmholtz-metadaten.de/cmso/SimulationCell 
    value: python id 140179083258256 
    units: None 
    data_type:  SampleSimulatio

In [84]:
wf_gui.children['CreateStructure'].__class__.node_function._semantikon_metadata['uri']().IRI

'https://id-from-pmdco-pending/StructureCreationNode'